In [ ]:
"""
This script retrieves the latest GTFS version from the Adelaide Metro API,
downloads the latest feed and a range of previous feed versions,
and extracts all ZIP files.
"""

In [1]:
import requests
import os
from io import BytesIO
from zipfile import ZipFile    

In [10]:
# Official page: https://gtfs.adelaidemetro.com.au/
# Base GTFS API URL
base_url = "http://gtfs.adelaidemetro.com.au/v1"

# Endpoint to get the latest GTFS version number
latest_version_number = "static/latest/version.txt"

# Binary data as response. For requests.get(), no special headers are required, so this variable is unused
# request_headers={"Content-Type": "application/octet-stream"}

# Template used to download a specific GTFS version ZIP
feed_url_template = "static/{version}/google_transit.zip"

# Output directories
dest_path = 'destination'
unzip_root = os.path.join(dest_path, "unzip")

# Create directories if they do not exist
os.makedirs(dest_path, exist_ok=True)
os.makedirs(unzip_root, exist_ok=True)

1. Download and unzip the latest GTFS feed ZIP

In [13]:
# 1. Retrieve the latest GTFS feed version number

# Define URLs and destination folder
latest_version_url = f"{base_url}/{latest_version_number}"
latest_feed_url_template = f"{base_url}/{feed_url_template}"

# Get latest feed version number
response = requests.get(latest_version_url)  
latest_version = response.text
print("Latest GTFS version:", latest_version)

# 2. Download latest GTFS feed version 
latest_feed_url = latest_feed_url_template.format(version=latest_version)  
print("Downloading GTFS feed from:", latest_feed_url)
response = requests.get(latest_feed_url) 

# Save the downloaded ZIP to the destination folder
latest_zip_path = os.path.join(dest_path, f"google_transit_{latest_version}.zip")
with open(latest_zip_path, "wb") as f:
    f.write(response.content)

print(f"Saved latest GTFS ZIP to: {latest_zip_path}")

# 3. Unzip the latest version into destination/unzip/latest/
latest_extract_path = os.path.join(unzip_root, "latest")
os.makedirs(latest_extract_path, exist_ok=True)

with ZipFile(latest_zip_path, "r") as zip_obj:
    zip_obj.extractall(latest_extract_path)

print(f"Extracted latest version to: {latest_extract_path}")

Latest GTFS version: 1606
Saved latest GTFS ZIP to: destination\google_transit_1606.zip
Extracted latest version to: destination\unzip\latest


2. Download and unzip versions 930–939

In [21]:
downloaded_versions = []

for version in range(930, 940):  
    version_feed_url = f"{base_url}/{feed_url_template.format(version=version)}"
    print(f"Downloading version {version} from: {version_feed_url}")
    
    response = requests.get(version_feed_url)
    
    if response.status_code == 200:
        zip_path = os.path.join(dest_path, f"google_transit_{version}.zip")        
        with open(zip_path, "wb") as f:
            f.write(response.content)
        print(f"Saved: {zip_path}")

        # Keep track of which versions we downloaded
        downloaded_versions.append(version)
        
    else:
        print(f"Version {version} not found (HTTP {response.status_code}).")

Saved: destination\google_transit_930.zip
Saved: destination\google_transit_931.zip
Saved: destination\google_transit_932.zip
Saved: destination\google_transit_933.zip
Saved: destination\google_transit_934.zip
Saved: destination\google_transit_935.zip
Saved: destination\google_transit_936.zip
Saved: destination\google_transit_937.zip
Saved: destination\google_transit_938.zip
Saved: destination\google_transit_939.zip


In [22]:
# Unzip specific versions
for version in downloaded_versions:
    zip_path = os.path.join(dest_path, f"google_transit_{version}.zip")
    extract_path = os.path.join(unzip_root, str(version))
    os.makedirs(extract_path, exist_ok=True)

    with ZipFile(zip_path, "r") as zip_obj:
        zip_obj.extractall(extract_path)

    print(f"Extracted version {version} to: {extract_path}")

Extracted version 930 to: destination\unzip\930
Extracted version 931 to: destination\unzip\931
Extracted version 932 to: destination\unzip\932
Extracted version 933 to: destination\unzip\933
Extracted version 934 to: destination\unzip\934
Extracted version 935 to: destination\unzip\935
Extracted version 936 to: destination\unzip\936
Extracted version 937 to: destination\unzip\937
Extracted version 938 to: destination\unzip\938
Extracted version 939 to: destination\unzip\939


3. Download and unzip versions 930–939 using BytesIO without saving the ZIPs to disk

In [23]:
for version in range(930, 940):
    version_feed_url = f"{base_url}/{feed_url_template.format(version=version)}"
    print(f"Downloading version {version} from: {version_feed_url}")

    response = requests.get(version_feed_url)

    if response.status_code == 200:
        print(f"Downloaded version {version} (size: {len(response.content)} bytes)")

        # Create extract folder for this version
        extract_path = os.path.join(unzip_root, str(version))
        os.makedirs(extract_path, exist_ok=True)

        # Use BytesIO instead of saving ZIP to disk ---
        zip_bytes = BytesIO(response.content)

        # Extract directly from memory
        with ZipFile(zip_bytes, "r") as zip_obj:
            zip_obj.extractall(extract_path)

        print(f"Extracted version {version} to: {extract_path}")

    else:
        print(f"Version {version} not found (HTTP {response.status_code}).")

Downloaded version 930 (size: 11876923 bytes)
Extracted version 930 to: destination\unzip\930
Downloaded version 931 (size: 11876998 bytes)
Extracted version 931 to: destination\unzip\931
Downloaded version 932 (size: 11781803 bytes)
Extracted version 932 to: destination\unzip\932
Downloaded version 933 (size: 11790409 bytes)
Extracted version 933 to: destination\unzip\933
Downloaded version 934 (size: 11738846 bytes)
Extracted version 934 to: destination\unzip\934
Downloaded version 935 (size: 11579431 bytes)
Extracted version 935 to: destination\unzip\935
Downloaded version 936 (size: 11584514 bytes)
Extracted version 936 to: destination\unzip\936
Downloaded version 937 (size: 11618342 bytes)
Extracted version 937 to: destination\unzip\937
Downloaded version 938 (size: 11618377 bytes)
Extracted version 938 to: destination\unzip\938
Downloaded version 939 (size: 18759739 bytes)
Extracted version 939 to: destination\unzip\939
